# 🎲 Searching for a “Better” Betting Strategy (Two Dice)

This program computes the winning probabilities **exactly**
$$ P_A(S,T),\ P_B(S,T) $$
and lists all betting strategies that are “better” than $S$, i.e., all $T$ such that
$$ P_B(S,T) > P_A(S,T). $$

In addition, the runtime and the total number of compared strategies are reported.

Since the number of possible betting strategies grows rapidly, optional bounds on the number of chips per field can be specified.

---

### ✅ Requirements

- Exactly **two dice** with user-defined outcome probabilities are used; the probabilities sum to 1.

---

### ⚙️ Inputs

- `p`: probabilities of the individual fields  
- `S`: base betting strategy  
- `min_limits`: minimum number of chips per field  
- `max_limits`: maximum number of chips per field  

---

### ▶️ Next Steps

The corresponding Python code is located in the **next code cell**.  
Only the following inputs need to be adjusted:

- `p`: probabilities of the fields  
  (e.g. `p = (1/2, 1/3, 1/6)`)
- `S`: base betting strategy  
  (e.g. `S = (3, 2, 1)`)
- `min_limits`: minimum number of chips per field  
  (e.g. `min_limits = (0, 0, 0)`)
- `max_limits`: maximum number of chips per field  
  (e.g. `max_limits = (6, 6, 6)`)


> 📌 Make sure that `p`, `S`, `min_limits`, and `max_limits` have the same length!

> 📌 Caution: The computation time increases rapidly.

---

### ▶️ Running the Program

- Click inside the code cell.
- Press `Shift + Enter` to start the computation.  
  Alternatively, click **▶ Run** in the toolbar at the top.
- Use **Run → Run All Cells** to execute all cells at once.

📎 Good luck experimenting with your own strategies!


In [1]:
import itertools
from functools import lru_cache
from typing import Tuple, List
import time

# ============================================================
# Input: base strategy and probabilities
# ============================================================


# ===== Example: "die hits" =====
S = (3, 2, 1)
p = (1/2, 1/3, 1/6)

# Define limits
min_limits = (0, 0, 0)
max_limits = (6, 6, 6)

# ===== Example: "difference hits" =====
# S = (3, 6, 5, 3, 1, 0)
# p = [3/18, 5/18, 4/18, 3/18, 2/18, 1/18]
# Define limits
# min_limits = (3, 5, 4, 3, 0, 0)
# max_limits = (4, 7, 5, 4, 2, 1)

# ============================================================
# Recursive functions
# ============================================================

def make_two_dice_recursion(p1_list: List[float], p2_list: List[float]):
    @lru_cache(maxsize=None)
    def P_A2(V: Tuple[int,...], W: Tuple[int,...]) -> float:
        V = list(V); W = list(W)
        if sum(V) == 0 and sum(W) > 0:
            return 1.0
        if sum(W) == 0:
            return 0.0

        X = 0.0
        p_stay = 0.0
        m = len(p1_list)
        for i in range(m):
            for j in range(m):
                pij = p1_list[i] * p2_list[j]
                a_rem = V[i] > 0
                b_rem = W[j] > 0
                if not (a_rem or b_rem):
                    p_stay += pij
                else:
                    V2 = V.copy(); W2 = W.copy()
                    if a_rem: V2[i] -= 1
                    if b_rem: W2[j] -= 1
                    s2, t2 = sum(V2), sum(W2)
                    if s2 == 0 and t2 > 0:
                        X += pij
                    elif t2 == 0:
                        pass
                    else:
                        X += pij * P_A2(tuple(V2), tuple(W2))
        return X / (1.0 - p_stay)

    return P_A2

def compute_two_dice(V: Tuple[int,...],
                     W: Tuple[int,...],
                     p1_list: List[float],
                     p2_list: List[float]):
    P_A2 = make_two_dice_recursion(p1_list, p2_list)
    PA = P_A2(tuple(V), tuple(W))
    PB = P_A2(tuple(W), tuple(V))
    PU = 1.0 - PA - PB
    return PA, PB, PU

# ============================================================
# New function with limits
# ============================================================

def generate_strategies_with_limits(total_chips: int,
                                    num_fields: int,
                                    min_limits: Tuple[int,...],
                                    max_limits: Tuple[int,...]):
    """
    Generates all strategies T (length num_fields, sum total_chips),
    such that for each field i: min_limits[i] <= T[i] <= max_limits[i].
    """
    for combo in itertools.combinations_with_replacement(range(num_fields), total_chips):
        strat = [0] * num_fields
        for idx in combo:
            strat[idx] += 1
        # Filter nach min/max
        valid = True
        for i in range(num_fields):
            if not (min_limits[i] <= strat[i] <= max_limits[i]):
                valid = False
                break
        if valid:
            yield tuple(strat)

# ============================================================
# Search for better strategies
# ============================================================

def find_better_strategies(S: Tuple[int,...],
                           p: List[float],
                           min_limits: Tuple[int,...],
                           max_limits: Tuple[int,...]):
    total = sum(S)
    m = len(S)
    better = []
    count = 0
    for T in generate_strategies_with_limits(total, m, min_limits, max_limits):
        count += 1
        PA, PB, _ = compute_two_dice(S, T, p, p)
        if PA < PB:
            better.append((T, PA, PB))
    return better, count

# ============================================================
# Run search
# ============================================================

t2 = time.perf_counter()
better_strats, total_comparisons = find_better_strategies(S, p, min_limits, max_limits)
t3 = time.perf_counter()

print(f"Runtime: {t3 - t2:.3f}s (two dice)")
print(f"Base strategy S = {S}")
print(f"Total strategies compared: {total_comparisons}")
print(f"Better strategies found: {len(better_strats)}\n")

for T, PA, PB in sorted(better_strats, key=lambda x: x[1]):
    T_str = str(T)
    print(f"T = {T_str:>4}  ⇒  P_A = {PA:.4f},  P_B = {PB:.4f}")


Runtime: 0.095s (two dice)
Base strategy S = (3, 2, 1)
Total strategies compared: 28
Better strategies found: 1

T = (4, 2, 0)  ⇒  P_A = 0.4378,  P_B = 0.4466
